In [1]:
import matplotlib.pyplot as plt
import cupy as cp

file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)

#
# make a matrix each column of which represents an images in a vector form 
#
images  = cp.empty((num_image, size_row * size_col), dtype=float)
labels  = cp.empty(num_image, dtype=int)

for cnt, line in enumerate(data):

    line_data   = line.split(',')
    label       = int(line_data[0])
    im_vector   = cp.array(list(map(int, line_data[1:])))

    labels[cnt]    = label
    images[cnt]    = im_vector

# Split Data
train_images = images[:6000]
train_labels = labels[:6000]
test_images = images[6000:]
test_labels = labels[:6000]

In [3]:
# Define module

class Linear:
    def __init__(self, in_shapes, out_shapes):
        self.weight = cp.random.normal(0, 1, (in_shapes, out_shapes))
        self.input = 0
    
    def __call__(self, x):
        self.input = x
        return self.weight.T @ x
    
    def backprop(self):
        return self.weight
        

def sigmoid(x):
    return 1 / (1 + cp.exp(-x))
    
class Sigmoid:
    output = 0
    
    def __call__(self, x):
        self.output = 1 / (1 + cp.exp(-x))
        return 1 / (1 + cp.exp(-x))
    
    def backprop(self):
        return self.output * (1 - self.output)
        
    
class Sequential:
    def __init__(self, modules: list):
        self.module_list = modules
    
    def __call__(self, x):
        for layer in self.module_list:
            x = layer(x)
            
        return x
    
    def backprop(self, preds, labels, criterion):
        s_val = criterion.backprop()

        for layer in reversed(self.module_list):
            if (type(layer) == type(Sigmoid)):
                s_val *= layer.backprop()
            else:
                layer.weight = s_val * layer.input
                s_val *= layer.backprop()
                
                del layer.input
                
            
        
class BinaryCrossEntropy:
    labels = 0
    preds = 0
    def __call__(self, preds, labels):
        self.labels = labels
        self.preds = preds
        
        return -(labals * cp.log(preds) + (1 - labels) * log(1 - preds))
    
    def backprop(self):
        return -self.labels / self.preds + (1 - self.labels) / (1 - self.preds)
    
class LearningRateScheduler:
    def __init__(self, init_lr, final_step, warm_up_step = 0):
        self.init_lr = init_lr
        self.final_step = final_step
        self.warm_up_step = warm_up_step
        self.lr = 0
        self.step = 0
    
    def step(self):
        step += 1
        if (step < self.warm_up_step):
            self.lr = (self.init_lr / warm_up_step) * step
        else:
            self.lr = (cp.cos(cp.pi * ((self.step - self.warm_up_step) / (final_step - self.warm_up_step))) + 1) * self.init_lr
        self.lr = cos
        
    def get_lr(self):
        return self.lr

In [4]:
model = Sequential([
    Linear(784, 196),
    Sigmoid(),
    Linear(196, 49),
    Sigmoid(),
    Linear(49, 10),
    Sigmoid()
])